In [5]:
#--------- Python Libraries ---------#

import requests
import json
import re
from html.parser import HTMLParser
from bs4 import BeautifulSoup
import csv
import pandas as pd
from datetime import date
import datetime
import dateutil.parser

#import spotipy
#import sys
#from spotipy.oauth2 import SpotifyClientCredentials

import SpotifyLookup

import string
import Levenshtein as lev

#SPOTIPY_CLIENT_ID = 'a5ad437675474f19bc6c76bb096dee85'
#SPOTIPY_CLIENT_SECRET = '69b402da629b41d498354ae80444c3d9'

#spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET))



In [6]:
#--------- Web crawling for URLs ---------#

source = requests.get('https://soundvenue.com/musik/anmeldelser').text
soup = BeautifulSoup(source, 'lxml')
print(source)

ConnectionError: HTTPSConnectionPool(host='soundvenue.com', port=443): Max retries exceeded with url: /musik/anmeldelser (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x10fbc18d0>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [ ]:
list_of_URLs = []

# Loop through all review links to find concert reviews specifically
for link in soup.find_all('div',{'class' : 'Spot-category'}):
    
    test = link.find('a')
    test2 = test.contents
    
    if 'Koncert' in test2[0]:
        print('Found concert review URL')
        URL = test.get('href')
        list_of_URLs.append(URL)
    else:
        continue

# Remove duplicate URLs in list
list_of_URLs = list(set(list_of_URLs))
print(list_of_URLs)

In [ ]:
# 2 test URLs
#list_of_URLs = ['https://soundvenue.com/musik/2020/08/poptalentet-dopha-overvandt-den-akavede-coronastemning-til-lillefredag-i-tivoli-415244']

# Create empty dictionary
critic_review_dict = {} 
critic_review_dict['concerts'] = []

# Loop - For every link in the URL_links do the following with the reviews:
for i in list_of_URLs:

    # Reset values
    critic_name = 'Soundvenue' 
    author = ''
    url = ''
    headline = ''
    text = ''
    rating = 0
    max_rating = 6
    tribute_rating = 0
    artist_name = ''
    venue_name = ''
    venue_place = ''
    publication_date = ''
    concert_date = ''
    lang = 'da'
    genre = []
    venue_is_online = False
    
    url = i
    
    # Get the source code of the webpage as text
    source = requests.get(i).text
    soup = BeautifulSoup(source, 'lxml')

    # Find headline
    try:
        headline = soup.h1.text
        print('Headline found')
    
    except:
        print('No headline found')
    
    # Find text
    try:
        text = soup.find('div',{'id' : 'content'})
        text = text.find_all('p')
        text = [re.sub(r'<.+?>',r'',str(a)) for a in text]
        text = '\n\n'.join(text)
        print('Text found')
    
    except:
        print('No text found')
    
    # Find artist name
    try:
        artist_name = soup.find('div',{'class' : 'facts'})
        artist_name = artist_name.find('b').contents
        artist_name = artist_name[0]
        artist_name = artist_name[:-1]
        print('artist name found')
    except:
        artist_name = ''
        print('No artist name found')
        continue
    
    # Find venue name
    try:
        venue_name = soup.find('div',{'class' : 'facts'})
        venue_name = venue_name.text
        #venue_name = venue_name.lstrip(artist_name + '. Koncert ')
        venue_name = venue_name.replace(artist_name, '', 1)
        venue_name = venue_name.replace('. Koncert. ', '', 1)
        venue_name = venue_name[:-1]
        print('venue name found')
        
        # Check if livestream koncert
        if venue_name.lower() == 'virtuel koncert':
            venue_is_online = True
            venue_name = 'Livestream'
            print('Livestream')
            
        
    except:
        print('No venue name found')
        continue
    
    
    # Find publication date
    try:    
        publication_date = soup.find('time')
        publication_date = publication_date.get('datetime')
        publication_date = dateutil.parser.parse(publication_date)
        print('publication date found')

    except:
        publication_date = ''
        print('No publication_date found') 
        continue
    
    # Find Author
    try:
        author = soup.find('span',{'itemprop' : 'author'})
        author = author.find('span',{'itemprop' : 'name'})
        author = author.find('a').text
        print('Author found')
        
    except:
        print('No author found')
    
    # Find rating
    try:
        rating = soup.find('div',{'class':'Article-rating'})
        rating = len(rating.find_all('div',{'class':'Icon Icon--star is-on'}))
        tribute_rating = (5/max_rating)*rating
        tribute_rating = round(tribute_rating, 2)
        print('Rating found')
    except:
        print('No rating found')
    
    """
    ########## Spotify lookup ############
    try:
        # Preprocessing: Remove all within () and punctuation, set lowercase
        preprocessed_name = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", artist_name)
        preprocessed_name = preprocessed_name.lower()
        preprocessed_name = preprocessed_name.translate(str.maketrans('', '', string.punctuation))
        print('Name preprocessing:', artist_name, '=' ,preprocessed_name)

        # Reset spotify values
        spotify_match = False
        artist_name_spotify = ""
        artist_img_spotify = ""
        artist_spotify_id = ""
        subgenres_spotify = []
        artist_popularity_spotify = ""

        # Make Spotify lookup on raw artist name
        results = spotify.search(q='artist:' + artist_name, type='artist')
        items = results['artists']['items']
        
        # If spotify returns no results on raw artist name search, make Spotify lookup on preprocessed artist name

        if len(items) == 0:
            print('No result on raw name, trying preprocessed name...')
            results = spotify.search(q='artist:' + preprocessed_name, type='artist')
            items = results['artists']['items']

        # If spotify returns results on raw artist name
        if len(items) > 0:

            print('found', len(items), 'results')
            print('searching for exact string match...')

            # First check all results for exact string match
            for ii in range(len(items)):
                artist = items[ii]

                # Set lower case and match songkick name with spotify name
                if artist_name.lower() == artist['name'].lower():

                    spotify_match = True
                    print('Found exact string match on index', ii, artist['name'])

                    artist_name_spotify = artist['name']
                    artist_img_spotify = artist['images'][0]['url']
                    artist_spotify_id = artist['id']
                    subgenres_spotify = artist['genres']
                    artist_popularity_spotify = artist['popularity']

                    break

                else:
                    print('No exact match on result from index ', ii, artist['name'])

            # If no match, then check all spotify results for fuzzy string match above 90%
            if spotify_match == False:
                print('searching for fuzzy string match...')

                for iii in range(len(items)):
                    artist = items[iii]

                    Distance = lev.distance(preprocessed_name,artist['name'].lower()),
                    Ratio = lev.ratio(preprocessed_name,artist['name'].lower())

                    if Ratio > 0.90:

                        spotify_match = True
                        print('Found fuzzy string match on index', iii, artist['name'])
                        print('with ratio:', Ratio)

                        artist_name_spotify = artist['name']
                        artist_img_spotify = artist['images'][0]['url']
                        artist_spotify_id = artist['id']
                        subgenres_spotify = artist['genres']
                        artist_popularity_spotify = artist['popularity']
                        break

                    else:
                        print('No fuzzy match on result from index ', iii, artist['name'])

        else:
            print('no Spotify search results at all from the preprocessed string')
    except:
        print('unsuccesful spotify lookup')

    """
    
    # Spotify Lookup with artist_name
    spotify_results = SpotifyLookup.spotify_lookup(artist_name)
    artist_name_spotify = spotify_results[0]
    artist_img_spotify = spotify_results[1]
    artist_spotify_id = spotify_results[2]
    subgenres_spotify = spotify_results[3]
    artist_popularity_spotify = spotify_results[4]
    
    print('Critic:', critic_name)
    print('Author:', author)
    print('URL:', url)
    print('Artist:', artist_name)
    print('Venue:', venue_name)
    print('Venue Place:', venue_place)
    print('Rating:', str(rating))
    print('Max Rating:', str(max_rating))
    print('Tribute Rating:', str(tribute_rating))
    print('Publication Date:', publication_date)
    print('Concert Date:', concert_date)
    print('artist spotify name:', artist_name_spotify)
    print('artist spotify image:', artist_img_spotify)
    print('artist spotify ID:', artist_spotify_id)
    print('artist spotify subgenres:', subgenres_spotify)
    print('artist spotify popularity:', artist_popularity_spotify)
    print('Language of review:', lang)
    print('Headline:', headline)
    print('Text:', text)
    
    # Set dictionary values

    temp_dict = {}
    temp_dict['critic_name'] = critic_name
    temp_dict['author'] = author
    temp_dict['url'] = url
    temp_dict['artist_name'] = artist_name
    temp_dict['artist_name_spotify'] = artist_name_spotify
    temp_dict['artist_img_spotify'] = artist_img_spotify
    temp_dict['artist_spotify_id'] = artist_spotify_id
    temp_dict['artist_popularity_spotify'] = artist_popularity_spotify
    temp_dict['genre'] = genre
    temp_dict['subgenre_spotify'] = subgenres_spotify
    temp_dict['publication_date'] = publication_date
    temp_dict['concert_date'] = concert_date
    temp_dict['venue_name'] = venue_name
    temp_dict['venue_place'] = venue_place
    temp_dict['rating'] = rating
    temp_dict['tribute_rating'] = tribute_rating
    temp_dict['max_rating'] = max_rating
    temp_dict['review_language'] = lang
    temp_dict['headline'] = headline
    temp_dict['text'] = text

    critic_review_dict['concerts'].append(temp_dict)

    
    
    


In [5]:
# Create json object with utf-8
with open('soundvenue_critic_reviews.json', 'w', encoding='utf-8') as json_file:
    json.dump(critic_review_dict, json_file, ensure_ascii=False)

In [118]:
with open('soundvenue_critic_reviews.json') as json_file:
    data = json.load(json_file)
    jstr = json.dumps(data, indent=4)
    print(jstr)


{
    "concerts": [
        {
            "critic_name": "Soundvenue",
            "author": "Kjartan F. Stolberg",
            "url": "https://soundvenue.com/musik/2020/08/poptalentet-dopha-overvandt-den-akavede-coronastemning-til-lillefredag-i-tivoli-415244",
            "artist_name": "Dopha",
            "artist_name_spotify": "Dopha",
            "artist_img_spotify": "https://i.scdn.co/image/bd51220ce9ddbe3ff4449bdfcab72c367e5d72d0",
            "artist_spotify_id": "7KNTK1VIkAqbjrFWGhYSwn",
            "artist_popularity_spotify": 30,
            "genre": [],
            "subgenre_spotify": [],
            "publication_date": "2020-08-14T10:39:45+02:00",
            "venue_name": "Tivoli",
            "venue_is_online": false,
            "rating": 4,
            "max_rating": 6,
            "review_language": "DK",
            "headline": "Poptalentet Dopha overvandt den akavede coronastemning til Lillefredag i Tivoli",
            "text": "S\u00e5 sad jeg der p\u00e5 en b\u00e